## PurpleAir Data
Data are available via PurpleAir's API. You will need to use a gmail account and create an API key via [this dashboard](https://develop.purpleair.com/sign-in?redirectURL=%2Fdashboards%2Fkeys). You should create a "Read" key that has a status of "Enabled". It's a good idea to add a label, host restrictions restrict the use of the key to certain machines, you do not need to set these. 

TODO: insert screen shot image

Once you have generated your key, you can "read" the key value to use in making requests. First, run the cell below and enter the key you generated when prompted. 

In [1]:
import getpass

api_key = getpass.getpass("Enter your API key: ")

Test your API key by running the code below. It should show the message "Key submit was successful" if your key is valid. 

In [8]:
import requests

url = "https://api.purpleair.com/v1/keys"

headers = {
    "X-API-Key": api_key 
}

response = requests.get(url, headers=headers)

if response.status_code == 201:
    print('Key submit was successful')
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")

Key submit was successful
{'api_version': 'V1.0.14-0.0.58', 'time_stamp': 1738866214, 'api_key_type': 'READ'}


If the API key is valid, a bounding box can be used to search for sensors. The coordinates in the cell below represent the longitude and latitude of the northwest and southeast corners of a box that encloses the South Bronx. The API request returns the identifiers of sensors within that bounding box. 

In [10]:
# corner latitude and longitudes in decimal degrees
nwlat = 40.833
nwlng = -73.933
selat = 40.783
selng = -73.783

url = 'https://api.purpleair.com/v1/sensors'

headers = {"X-API-Key": api_key}

params = {
    'fields':'name,last_seen',
    'location_type':0,

}

with requests.get(url, headers=headers, params=params) as response:

    if response.status_code == 200:
        print('Success')
        data = response.json()
        print(data)
    else:
        print(f"Request failed with status code: {response.status_code}")


Success
{'api_version': 'V1.0.14-0.0.58', 'time_stamp': 1738869490, 'data_time_stamp': 1738869473, 'location_type': 0, 'max_age': 604800, 'firmware_default_version': '7.02', 'fields': ['sensor_index', 'last_seen', 'name'], 'data': [[262259, 1738869407, 'PurpleAir91107'], [262265, 1738869383, 'Highlands Ranch'], [182, 1738869399, 'Jolly Brothers Road, Gabriola Island BC P1'], [195, 1738869374, 'Westhaven Dr N, Trinidad'], [286, 1738869371, 'Lasqueti Island'], [314, 1738869449, 'Parker'], [322, 1738867177, 'Hemberg Dr'], [324, 1738857675, 'Mitsue-Mura Nara Japan'], [334, 1738833755, 'Moody Ave'], [340, 1738869466, 'Ingram Road'], [344, 1738869403, 'Sylvan Crescent'], [387, 1738869443, 'Monmouth Drive P1'], [443, 1738869470, 'Weber-Morgan Health Department P1'], [453, 1738869380, 'LRAPA-Oakridge City Hall'], [459, 1738612057, 'Flight Park North 2'], [473, 1738869456, 'Sunnyside 93727'], [547, 1738869441, 'AQMD_RTI_1'], [661, 1738869458, 'Clean Air Carolina Coulwood West'], [747, 173886935

In [16]:
len(data['data'])

20565